In [ ]:
pip install xmltodict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from urllib import parse
import xmltodict
import requests
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from urllib.parse import unquote

In [ ]:
def get_request_query(url, serviceKey, params, gubun):
    import urllib.parse as urlparse
    params = urlparse.urlencode(params)
    request_query = url + "?serviceKey=" + serviceKey + '&' + params + '&gubun=' + gubun 
    return request_query

Seoul_data = pd.DataFrame() # 최종데이터를 받을 데이터프레임

In [ ]:
start_date = '20200729' # 2020년 7월 29일 ~ (차분 후 8월 데이터를 모두 사용하기 위함)
end_date = '20230305' # ~ 3월 5일까지 데이터 존재
date_list = [d.strftime('%Y-%m-%d') for d in pd.date_range(start=start_date, end=end_date, freq="D")]

In [ ]:
url_base = "http://apis.data.go.kr/1352000/ODMS_COVID_04/callCovid04Api"
url_serviceKey = "2qbQVBtKT9B0MU%2BONommVKsHJNUzNczJOH3DqVI8CIETW28uJqTgK9SRiXRBCau0%2Fr8oeVltLeOwE%2BePIxQujg%3D%3D"
# url_pages = "1" #페이지당열갯수
url_gubun = '서울' #지역

urls = []
for date in date_list:
    params = {'pageNo' : '1', 'numOfRows' : '500', 'std_day' : date}
    urls.append(get_request_query(url_base, url_serviceKey, params, url_gubun))

In [ ]:
dict_data = []

for url in tqdm(urls):
    req = requests.get(url).content
    xmlObject = xmltodict.parse(req)
    dict_data.append(xmlObject['response']['body']['items']['item'])

100%|██████████| 948/948 [1:16:25<00:00,  4.84s/it]


In [ ]:
concat_df = pd.DataFrame()
for data in dict_data:
    if type(data) == list:
        df = pd.DataFrame([data[0]])
        concat_df = pd.concat([concat_df, df], ignore_index=True)
    else:
        df = pd.DataFrame([data])
        concat_df = pd.concat([concat_df, df], ignore_index=True)

concat_df

,deathCnt,defCnt,gubun,gubunCn,gubunEn,incDec,isolClearCnt,isolIngCnt,localOccCnt,overFlowCnt,qurRate,stdDay
0,11,1600,서울,首尔,Seoul,8,1423,166,8,0,16.44,2020-07-31
1,11,1602,서울,首尔,Seoul,2,1433,158,2,0,16.46,2020-08-01
2,11,1607,서울,首尔,Seoul,5,1441,155,5,0,16.51,2020-08-02
3,11,1612,서울,首尔,Seoul,5,1448,153,2,3,16.56,2020-08-03
4,11,1621,서울,首尔,Seoul,9,1461,149,6,3,16.65,2020-08-04
...,...,...,...,...,...,...,...,...,...,...,...,...
943,6345,5926866,서울,首尔,Seoul,2301,0,0,2301,0,62862,2023-03-01
944,6347,5928066,서울,首尔,Seoul,1200,0,0,1199,1,62875,2023-03-02
945,6351,5930362,서울,首尔,Seoul,2296,0,0,2292,4,62899,2023-03-03
946,6352,5932505,서울,首尔,Seoul,2214,0,0,2214,0,62922,2023-03-04


In [42]:
use_df = concat_df[['gubun','defCnt','stdDay']]
col =['시도명','def_Acc','date']
use_df.columns = col
use_df.set_index('date', drop=True, inplace=True)

In [57]:
use_df

,시도명,def_Acc
date,,
2020-07-29,서울,1589
2020-07-30,서울,1592
2020-07-31,서울,1600
2020-08-01,서울,1602
2020-08-02,서울,1607
...,...,...
2023-03-01,서울,5926866
2023-03-02,서울,5928066
2023-03-03,서울,5930362


In [58]:
use_df.to_csv(f'./drive/MyDrive/{url_gubun}_data.csv')

In [59]:
data = pd.read_csv(f'./drive/MyDrive/{url_gubun}_data.csv', index_col=['date'])
data

,시도명,def_Acc
date,,
2020-07-29,서울,1589
2020-07-30,서울,1592
2020-07-31,서울,1600
2020-08-01,서울,1602
2020-08-02,서울,1607
...,...,...
2023-03-01,서울,5926866
2023-03-02,서울,5928066
2023-03-03,서울,5930362


In [60]:
data['daily_Acc'] = data['def_Acc'].diff()
data['DAC'] = data['daily_Acc'].diff()
data['DDAC'] = data['DAC'].diff()
# data.drop(data.index[:3], inplace=True)
data

,시도명,def_Acc,daily_Acc,DAC,DDAC
date,,,,,
2020-07-29,서울,1589,NaN,NaN,NaN
2020-07-30,서울,1592,3.0,NaN,NaN
2020-07-31,서울,1600,8.0,5.0,NaN
2020-08-01,서울,1602,2.0,-6.0,-11.0
2020-08-02,서울,1607,5.0,3.0,9.0
...,...,...,...,...,...
2023-03-01,서울,5926866,2301.0,-25.0,-1728.0
2023-03-02,서울,5928066,1200.0,-1101.0,-1076.0
2023-03-03,서울,5930362,2296.0,1096.0,2197.0


In [61]:
data.to_csv(f'./drive/MyDrive/{url_gubun}_data.csv')